<a href="https://colab.research.google.com/github/AntonPrazdnichnykh/TTS_HW/blob/master/HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

В этом задании вам предстоит сделать так, чтобы TTS начал говорить другим голосом так, чтобы можно было контролировать интонации путем присвоения токена каждому слову.

**Важно!** Добавьте ссылку на свой форк с гитхаба в команду git clone

In [ ]:
!git clone https://github.com/AntonPrazdnichnykh/TTS_HW.git
!cd TTS_HW && curl gdrive.sh | bash -s https://drive.google.com/file/d/1ubAsUdWqHFY3MZJZVLqzxjH6lR_hKRyz/view?usp=sharing 
!cd TTS_HW && tar -xf libri_training_data.tar.gz


Cloning into 'TTS_HW'...
remote: Enumerating objects: 278, done.
remote: Counting objects: 100% (278/278), done.
remote: Compressing objects: 100% (211/211), done.
remote: Total 278 (delta 71), reused 266 (delta 63), pack-reused 0
Receiving objects: 100% (278/278), 4.72 MiB | 15.40 MiB/s, done.
Resolving deltas: 100% (71/71), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2874  100  2874    0     0   5283      0 --:--:-- --:--:-- --:--:--  5273
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3287    0  3287    0     0  26942      0 --:--:-- --:--:-- --:--:-- 26942
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   

## Важно!

Следующая ячейка будет устанавливать библиотеку Apex, на колабе это может занять **12-20** минут! К сожалению, без этой библиотеки обучение данной реализации сети будет длиться слишком медленно, также из этой библиотеки используется оптимайзер FusedLamb, если же использовать стандартный Adam, обучение может стать нестабильным.

Если у вас есть своя локальная GPU, рекомендую использовать docker контейнер с Nvidia GPU Cloud - `nvcr.io/nvidia/pytorch:21.06-py3`, там эта библиотека уже предустановлена.

In [ ]:
try:
  import apex
except Exception:
  ! git clone https://github.com/NVIDIA/apex.git
  % cd apex
  !pip install --target=$nb_path --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
  %cd ..

Cloning into 'apex'...
remote: Enumerating objects: 8681, done.
remote: Counting objects: 100% (1071/1071), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 8681 (delta 961), reused 846 (delta 846), pack-reused 7610
Receiving objects: 100% (8681/8681), 14.35 MiB | 20.76 MiB/s, done.
Resolving deltas: 100% (5938/5938), done.
/content/apex
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Processing /content/apex
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/

In [ ]:
!pip install -r TTS_HW/requirements.txt
!pip install transformers
!pip install torchmetrics

  Cloning git://github.com/NVIDIA/dllogger.git (to revision 26a0f8f1958de2c0c460925ff6102a4d2486d6cc) to /tmp/pip-install-p0c17b19/dllogger_cf8f230a7dd24e638c98714621e6f7f1
  Running command git clone -q git://github.com/NVIDIA/dllogger.git /tmp/pip-install-p0c17b19/dllogger_cf8f230a7dd24e638c98714621e6f7f1
  Running command git rev-parse -q --verify 'sha^26a0f8f1958de2c0c460925ff6102a4d2486d6cc'
  Running command git fetch -q git://github.com/NVIDIA/dllogger.git 26a0f8f1958de2c0c460925ff6102a4d2486d6cc
  Running command git checkout -q 26a0f8f1958de2c0c460925ff6102a4d2486d6cc
     |████████████████████████████████| 183 kB 5.5 MB/s 
     |████████████████████████████████| 195 kB 39.8 MB/s 
  Created wheel for dllogger: filename=DLLogger-0.1.0-py3-none-any.whl size=5630 sha256=b84728041ed44f4109aa95af03186df35826233b5d2ee8b630a34635f5957524
  Stored in directory: /root/.cache/pip/wheels/69/76/a4/ae417dad079ab1cae58eb65601ec01d980c5e02915f4f1792f
  Created wheel for librosa: filename=lib

In [ ]:
!cd TTS_HW && bash scripts/download_cmudict.sh
!cd TTS_HW && bash scripts/download_fastpitch.sh
!cd TTS_HW && bash scripts/download_waveglow.sh

Extracting nvidia_fastpitch_210824.pt ...
OK
Extracting nvidia_waveglow256pyt_fp16.pt ...
OK
































### Описание задачи
Выше была склонирована оригинальная имплементация FastPitch [отсюда](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/SpeechSynthesis/FastPitch). В ней изменен лишь даталоадер - в батче помимо текста и целевой мелспектрограммы, также возвращаются токены `Prominencе`. Необходимо понять, как можно использовать эти токены и в какое место сети необходимо их добавить, чтобы затем, на инференсе, можно было контролировать интонации.

Так как мы сильно ограничены в ресурсах - обучать сеть с нуля не получится, необходимо лишь провести finetuning с оригинального чекпоинта от Nvidia, обученного на датасете LJSpeech. От вас требуется дообучить сеть на одном из голосов из датасета LibriTTS.

Все данные уже предобработаны и загружены, токены `Prominence` были извлечены с помощью [Wavelet Prosody Toolkit](https://github.com/asuni/wavelet_prosody_toolkit). Эти значения были квантизированы, в данных могут встречаться лишь значения 1, 2, 3. Эти значения извлечены для каждого слова и продублированы для каждой фонемы.

Из-за особенностей имплементации засовывать обучение в ноутбук крайне неудобно, поэтому предлагается изменить файлы исходного кода прямо открывая их в колабе (они видны в файл менеджере слева), а затем запустить обучение, выполнив ячейку ниже. Чтобы не потерять свои изменения, рекомендую, изменив файлы, просто коммитить их к себе в репозиторий прямо из колаба. Не забывайте это делать часто, так как колаб любит уничтожать вашу среду, даже если вы просто отошли попить кофе.

Уже 10 эпох обучения достаточно (около 15 минут обучения), чтобы была слышна разница. Mel loss должен опуститься где-то до значений 0.2-0.3, чтобы TTS звучал нормально. Так как данных очень мало, то если вы оставите сеть учиться надолго, скорее всего она переобучится, поэтому учить сильно дольше 20 эпох нет смысла.

In [ ]:
!pwd

In [ ]:
%cd ../

In [ ]:
!cd TTS_HW && bash scripts/train.sh



AMP=true, 1x5x2 (global batch size 10)

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
train.py:43: UserWarning: PyProf is unavailable
  warnings.warn('PyProf is unavailable')
DLL 2021-12-04 20:27:40.637713 - PARAMETER | output :  ./output
DLL 2021-12-04 20:27:40.638013 - PARAMETER | dataset_path :  libri_training_data
DLL 2021-12-04 20:27:40.638094 - PARAMETER | log_file :  ./output/nvlog.json
DLL 2021-12-04 20:27:40.638200 - PARAMETER | pyprof :  False
DLL 2021-12-04 20:27:40.638298 - PARAMETER | epochs :  10
DLL 2021-12-04 20:27:40.638410 - PARAMETER | epochs_per_ch

После того как обучили сеть, прогоните ее на одной заданной фразе с разными значениями `Prominence`. Для этого в файле `inference.py` измените 384 строчку и запустите ячейку ниже. С разными значениями prominence фраза "What a day" должна произноситься с разной интонацией. Если вы этого добились, значит задание выполнено.

In [ ]:
!cd TTS_HW && bash scripts/inference_example.sh


AMP=false, batch_size=32

DLL 2021-12-04 20:45:45.771839 - PARAMETER | input :  phrases/phrase_1_64.txt
DLL 2021-12-04 20:45:45.771941 - PARAMETER | output :  ./output/audio_phrase_1_64.txt
DLL 2021-12-04 20:45:45.772016 - PARAMETER | log_file :  ./output/audio_phrase_1_64.txt/nvlog_infer.json
DLL 2021-12-04 20:45:45.772128 - PARAMETER | save_mels :  False
DLL 2021-12-04 20:45:45.772198 - PARAMETER | cuda :  True
DLL 2021-12-04 20:45:45.772271 - PARAMETER | cudnn_benchmark :  True
DLL 2021-12-04 20:45:45.772359 - PARAMETER | fastpitch :  output/FastPitch_checkpoint_10.pt
DLL 2021-12-04 20:45:45.772440 - PARAMETER | waveglow :  pretrained_models/waveglow/nvidia_waveglow256pyt_fp16.pt
DLL 2021-12-04 20:45:45.772516 - PARAMETER | sigma_infer :  0.9
DLL 2021-12-04 20:45:45.772583 - PARAMETER | denoising_strength :  0.01
DLL 2021-12-04 20:45:45.772667 - PARAMETER | sampling_rate :  22050
DLL 2021-12-04 20:45:45.772755 - PARAMETER | stft_hop_length :  256
DLL 2021-12-04 20:45:45.772825 - PA

In [ ]:
import IPython
IPython.display.Audio('/content/TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

In [ ]:
import IPython
IPython.display.Audio('/content/TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

# Задание 2 (5 баллов)

Это хорошо, что мы научились контролировать интонацию в произносимых фразах, но что делать на инференсе, ведь у нас нет этих значений `Prominence`? Можно попробовать научиться предсказывать их из текста. Для этого можно обучить какую-нибудь LM, при этом учить с нуля также не имеет смысла.

В задании предлагается заполнить отсутствующие ячейки ниже. Необходимый для зачета F1 score на тестовой выборке должен быть около `0.75`. Не забудьте про маски!

In [ ]:
import torch
import torch.nn.functional as F
import transformers
import os
from pathlib import Path
import numpy as np
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel
from tqdm import tqdm
import torchmetrics

device='cuda:0'

In [ ]:
def val_to_class(val):
    if val < 1:
        return 0
    if 1 < val < 2:
        return 1
    return 2

f1_criterion = torchmetrics.F1(num_classes=3)
acc_criterion = torchmetrics.Accuracy(num_classes=3)

class PromDataset(Dataset):
    def __init__(self, tokenizer, sentences, labels, max_len, extend=True):
        self.len = len(sentences)
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self._extend = extend
        
    def __getitem__(self, index):
        #####################
        tokens = self.tokenizer.tokenize(self.sentences[index])
        words = self.sentences[index].split()
        word_labels = self.labels[index]
        if self._extend:
          label = [0] # cls token
          for word, lbl in zip(words, word_labels):
            label.extend([lbl] * len(self.tokenizer.tokenize(word)))
          label.append(0)  # sep token
        else:
          label = word_labels
        
        ids = self.tokenizer.encode(self.sentences[index])[:self.max_len]
        label = label[:self.max_len]
        l_ids = len(ids)
        assert len(label) == l_ids, "labels and tokens must be of the same length"
        mask = [1] * l_ids
        if l_ids < self.max_len:
          diff = self.max_len - l_ids
          ids.extend([0] * diff)
          label.extend([0] * diff)
          mask.extend([0] * diff)
        #####################
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'tags': torch.tensor(label, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

sentences = []
label_proms = []
label_boundaries = []
for filename in Path('TTS_HW/libri_training_data').glob('*.prom'):
    #####################
    with open(filename, 'r') as f:
      words = []
      proms = []
      boundaries = []
      for line in f:
        line = line.rstrip('\n')
        _, _, _, word, prom, _ = line.split('\t')
        words.append(word if word != '<unk>' else '[UNK]')
        proms.append(val_to_class(float(prom)))
        sentences.append(" ".join(words))
        label_proms.append(proms)
        label_boundaries.append(" ".join(boundaries))
    #####################


tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
train_dataset = PromDataset(tokenizer, sentences[:-10], label_proms[:-10], 200)
test_dataset = PromDataset(tokenizer, sentences[-10:], label_proms[-10:], 200)

training_loader = DataLoader(train_dataset, batch_size=24, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [ ]:
batch = next(iter(training_loader))

In [ ]:
batch

{'ids': tensor([[ 101, 1133, 3229,  ...,    0,    0,    0],
         [ 101, 1501, 1169,  ...,    0,    0,    0],
         [ 101, 1128, 1138,  ...,    0,    0,    0],
         ...,
         [ 101,  178, 1189,  ...,    0,    0,    0],
         [ 101, 1103, 4809,  ...,    0,    0,    0],
         [ 101, 1296, 1937,  ...,    0,    0,    0]]),
 'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'tags': tensor([[0, 0, 1,  ..., 0, 0, 0],
         [0, 1, 1,  ..., 0, 0, 0],
         [0, 1, 1,  ..., 0, 0, 0],
         ...,
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 2,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0]])}

In [ ]:
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=3)
model.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=2e-05)
for epoch in range(10):
    print(f"EPOCH:{epoch}")
    t_bar = tqdm(training_loader)
    model.train()
    for batch in t_bar:
        batch_size = batch['ids'].size(0)
        optimizer.zero_grad()
        #####################

        batch['ids'] = batch['ids'].to(device)
        batch['mask'] = batch['mask'].to(device).float()
        batch['tags'] = batch['tags'].to(device)

        outputs = model(
            input_ids=batch['ids'],
            attention_mask=batch['mask'],
            labels=batch['tags']
            )
        
        # loss = outputs.loss
        loss = F.cross_entropy(outputs.logits.transpose(1, 2), batch['tags'], reduction='none')
        loss = loss * batch['mask']
        loss = loss.sum() / batch_size
        loss.backward()
        #####################
        t_bar.set_description(desc=f'loss; {loss.item()}')
        optimizer.step()

    model.eval()
    with torch.inference_mode():
        accs = []
        f1_scores = []

        for batch in test_loader:
          #####################
          outputs = model(
              input_ids=batch['ids'].to(device),
              attention_mask=batch['mask'].float().to(device),
              # labels=batch['tags'].to(device)
              )
          
          logits = outputs.logits.cpu()
          preds = logits.argmax(-1)
          # print('target:', batch['tags'])
          # print('pred:', preds)
          accs.append(
              acc_criterion(
                  preds[batch['mask'] == 1], batch['tags'][batch['mask'] == 1]
                  )
              )
          f1_scores.append(
              f1_criterion(
                  preds[batch['mask'] == 1], batch['tags'][batch['mask'] == 1]
                  )
              )
          #####################
            

    print(f"Accuracy: {torch.mean(torch.stack(accs)).item()}")
    print(f"F1 score: {torch.mean(torch.stack(f1_scores)).item()}")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

EPOCH:0


loss; 1.4174376726150513: 100%|██████████| 246/246 [06:11<00:00,  1.51s/it]


Accuracy: 0.6315789222717285
F1 score: 0.6315789222717285
EPOCH:1


loss; 1.4085259437561035: 100%|██████████| 246/246 [06:12<00:00,  1.51s/it]


...

Accuracy: 0.7894737124443054
F1 score: 0.7894737124443054
EPOCH:8


loss; 0.020655810832977295: 100%|██████████| 246/246 [06:11<00:00,  1.51s/it]


Accuracy: 0.8105263113975525
F1 score: 0.8105263113975525
EPOCH:9


loss; 0.004111329559236765: 100%|██████████| 246/246 [06:11<00:00,  1.51s/it]


Accuracy: 0.8105263113975525
F1 score: 0.8105263113975525


Стоит отметить, что мы обучаем сеть на очень небольшом наборе данных - если же файнтюнить на полном ТТС датасете, то точность предсказаний меток может достичь 0.95, поэтому такую сеть можно использовать при реальном инференсе для более точного предсказания интонаций.

# Задание 3 (5 баллов) (опционально)
Если по какой-то причине что-то не получилось в предыдущих заданиях, то можно попробовать подготовить модель к инференсу. Задача - заэкспортировать код модели в ONNX, и проверить, что он работает.
Подсказка - вся модель скорее всего не заэкспортится сразу, стоит пробовать делать это по модулям, а если какая-то функция не поддерживается ONNX, то ее можно выделить отдельно и заэкспортировать, например, в TorchScript. На выходе ожидается, что есть набор onnx или torchscript модулей, выполнив последовательно которые можно получить тот же выход, что и при запуске модели на PyTorch.

Задание - по желанию, если не хочется делать что-то из того, что выше!